## Setup

In [4]:
%pip install --upgrade --quiet --user google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 51.1 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## Environment variables

In [5]:
# get project ID
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"
if PROJECT_ID == "(unset)":
    print(f"Please set the project ID manually below")

Please set the project ID manually below


In [6]:
# define project information
if PROJECT_ID == "(unset)":
    PROJECT_ID = "gen-lang-client-0974620078"  # @param {type:"string"}

# generate an unique id for this session
from datetime import datetime

UID = datetime.now().strftime("%m%d%H%M")

## Authentication (Colab only)
If you are running this notebook on Colab, you will need to run the following cell authentication. This step is not required if you are using Vertex AI Workbench as it is pre-authenticated.

In [7]:
import sys

# if it's Colab runtime, authenticate the user with Google Cloud
if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

## Create and update item to GCS

### Get a sparse embedding

In [11]:
import pandas as pd
df = pd.read_json('/content/L21_V001_text.json')

### Dense embedding

In [35]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModel
import requests
import numpy as np
import torch

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Qwen/Qwen3-Embedding-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_dense_embedding(text: str):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        last_hidden_state = outputs.last_hidden_state  # (batch, seq_len, hidden)

        # Attention mask để bỏ padding
        mask = inputs["attention_mask"].unsqueeze(-1).expand(last_hidden_state.size())
        masked_embeddings = last_hidden_state * mask

        # Mean pooling
        sum_embeddings = masked_embeddings.sum(dim=1)
        sum_mask = mask.sum(dim=1)
        embeddings = sum_embeddings / sum_mask

        emb = embeddings[0].cpu().numpy().tolist()
    return emb

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

In [12]:
# Hàm chuyển số thứ tự sang chữ tiếng Việt
def number_to_vietnamese_order(n):
    units = ["", "một", "hai", "ba", "bốn", "năm", "sáu", "bảy", "tám", "chín"]
    tens = ["", "mười", "hai mươi", "ba mươi", "bốn mươi", "năm mươi",
            "sáu mươi", "bảy mươi", "tám mươi", "chín mươi"]

    if n <= 10:
        first10 = ["thứ nhất", "thứ hai", "thứ ba", "thứ tư", "thứ năm",
                   "thứ sáu", "thứ bảy", "thứ tám", "thứ chín", "thứ mười"]
        return first10[n-1]

    ten = n // 10
    unit = n % 10

    if ten == 1:
        text = "mười"
    else:
        text = tens[ten]

    if unit == 0:
        order_text = f"thứ {text}"
    elif unit == 1 and ten > 1:
        order_text = f"thứ {text} mốt"
    elif unit == 5:
        order_text = f"thứ {text} lăm"
    else:
        order_text = f"thứ {text} {units[unit]}"

    return order_text


In [13]:
OBJ_NAME_MAP = {
    # Con người & giới tính
    "Person": "Người",
    "Man": "Đàn ông",
    "Woman": "Phụ nữ",
    "Boy": "Cậu bé",
    "Girl": "Cô bé",
    "Baby": "Em bé",

    # Động vật
    "Bird": "Chim",
    "Cat": "Mèo",
    "Dog": "Chó",
    "Horse": "Ngựa",
    "Sheep": "Cừu",
    "Cow": "Bò",
    "Elephant": "Voi",
    "Bear": "Gấu",
    "Zebra": "Ngựa vằn",
    "Giraffe": "Hươu cao cổ",
    "Rabbit": "Thỏ",
    "Fox": "Cáo",
    "Panda": "Gấu trúc",
    "Tiger": "Hổ",
    "Lion": "Sư tử",
    "Deer": "Hươu",
    "Monkey": "Khỉ",
    "Snake": "Rắn",
    "Frog": "Ếch",
    "Turtle": "Rùa",
    "Whale": "Cá voi",
    "Dolphin": "Cá heo",

    # Phương tiện giao thông
    "Bicycle": "Xe đạp",
    "Bicycle wheel": "Bánh xe đạp",
    "Bicycle helmet": "Mũ bảo hiểm xe đạp",
    "Car": "Ô tô",
    "Motorcycle": "Xe máy",
    "Airplane": "Máy bay",
    "Bus": "Xe buýt",
    "Train": "Tàu hỏa",
    "Truck": "Xe tải",
    "Boat": "Thuyền",
    "Ship": "Tàu lớn",
    "Helicopter": "Trực thăng",
    "Scooter": "Xe tay ga",
    "Subway": "Tàu điện ngầm",

    # Nhà cửa & kiến trúc
    "Skyscraper": "Tòa nhà chọc trời",
    "Building": "Tòa nhà",
    "House": "Ngôi nhà",
    "Apartment": "Căn hộ",
    "Castle": "Lâu đài",
    "Bridge": "Cầu",
    "Tower": "Tháp",
    "Road": "Đường",
    "Street": "Phố",
    "Parking lot": "Bãi đỗ xe",
    "Playground": "Sân chơi",

    # Cảnh vật thiên nhiên
    "Beach": "Bãi biển",
    "Mountain": "Núi",
    "River": "Sông",
    "Lake": "Hồ",
    "Forest": "Rừng",
    "Desert": "Sa mạc",
    "Waterfall": "Thác nước",
    "Volcano": "Núi lửa",
    "Cave": "Hang động",
    "Island": "Đảo",
    "Glacier": "Sông băng",

    # Cây cối, hoa quả, rau củ
    "Tree": "Cây",
    "Palm tree": "Cây cọ",
    "Pine tree": "Cây thông",
    "Flower": "Hoa",
    "Rose": "Hoa hồng",
    "Sunflower": "Hoa hướng dương",
    "Tulip": "Hoa tulip",
    "Cactus": "Xương rồng",
    "Banana": "Chuối",
    "Apple": "Táo",
    "Orange": "Cam",
    "Lemon": "Chanh",
    "Strawberry": "Dâu tây",
    "Watermelon": "Dưa hấu",
    "Tomato": "Cà chua",
    "Carrot": "Cà rốt",
    "Broccoli": "Bông cải xanh",
    "Potato": "Khoai tây",
    "Onion": "Hành tây",

    # Đồ chơi
    "Teddy bear": "Gấu bông",
    "Ball": "Bóng",
    "Puzzle": "Trò chơi ghép hình",
    "Doll": "Búp bê",
    "Toy car": "Xe đồ chơi",
    "Lego": "Lego",
    "Yo-yo": "Con quay",
    "Frisbee": "Đĩa bay",

    # Dụng cụ gia đình, đồ điện tử
    "Chair": "Ghế",
    "Couch": "Sofa",
    "Table": "Bàn",
    "Lamp": "Đèn",
    "Fan": "Quạt",
    "Refrigerator": "Tủ lạnh",
    "Microwave": "Lò vi sóng",
    "Oven": "Lò nướng",
    "Toaster": "Máy nướng bánh",
    "Sink": "Bồn rửa",
    "Washing machine": "Máy giặt",
    "Vacuum cleaner": "Máy hút bụi",
    "Laptop": "Máy tính xách tay",
    "Desktop computer": "Máy tính để bàn",
    "Printer": "Máy in",
    "Camera": "Máy ảnh",
    "Phone": "Điện thoại",
    "TV": "Tivi",
    "Remote": "Điều khiển",

    # Quần áo & phụ kiện
    "Backpack": "Ba lô",
    "Handbag": "Túi xách",
    "Tie": "Cà vạt",
    "Suitcase": "Vali",
    "Hat": "Mũ",
    "Glasses": "Kính",
    "Sunglasses": "Kính mát",
    "Shoes": "Giày",
    "Coat": "Áo khoác",
    "Dress": "Váy",
    "Skirt": "Chân váy",
    "Shorts": "Quần short",
    "Pants": "Quần dài",
    "Shirt": "Áo sơ mi",
    "T-shirt": "Áo phông",
    "Jacket": "Áo khoác",

    # Thể thao & thực phẩm
    "Baseball bat": "Gậy bóng chày",
    "Baseball glove": "Găng tay bóng chày",
    "Skateboard": "Ván trượt",
    "Surfboard": "Ván lướt sóng",
    "Tennis racket": "Vợt tennis",
    "Bowl": "Bát",
    "Cup": "Cốc",
    "Fork": "Nĩa",
    "Knife": "Dao",
    "Spoon": "Thìa",
    "Pizza": "Pizza",
    "Cake": "Bánh ngọt",
    "Donut": "Bánh vòng",
    "Sandwich": "Bánh mì kẹp",
    "Bottle": "Chai",
    "Wine glass": "Ly rượu",

    # Vật dụng & công cụ
    "Skis": "Ván trượt tuyết",
    "Snowboard": "Ván trượt",
    "Umbrella": "Dù",

    # Đèn & ánh sáng
    "Lamp": "Đèn",
    "Ceiling lamp": "Đèn trần",
    "Street lamp": "Đèn đường",
    "Lantern": "Đèn lồng",
    "Flashlight": "Đèn pin",
    "Neon sign": "Bảng đèn neon",
    "Chandelier": "Đèn chùm",
    "Candle": "Nến"

}

COLOR_MAP = {
    "Gray": "Xám",
    "Black": "Đen",
    "White": "Trắng",
    "Red": "Đỏ",
    "Blue": "Xanh dương",
    "Green": "Xanh lá",
    "Yellow": "Vàng",
    "Brown": "Nâu",
    "Orange": "Cam",
    "Pink": "Hồng",
    "Purple": "Tím"
}

In [22]:
from collections import Counter
import json

def build_text_for_embedding(metadata, img_w=1.0, img_h=1.0):
    # Check if 'objects' is a dictionary of counts
    objects_data = metadata.get("objects", {})

    if isinstance(objects_data, dict):
        # If it's a dictionary of counts, format it directly
        if not objects_data:
            objects_text = ""
        else:
            objects_text = ", ".join([f"{OBJ_NAME_MAP.get(name, name)}: {count}" for name, count in objects_data.items()])

    elif isinstance(objects_data, list):
        # If it's a list (for backward compatibility or other data), process as before
        objs = [o for o in objects_data if isinstance(o, dict)]
        if not objs:
            objects_text = ""
            print("Warning: 'objects' list contains no valid dictionaries.")
        else:
            names_mapped = []
            for o in objs:
                if isinstance(o, dict):
                    names_mapped.append(OBJ_NAME_MAP.get(o.get("name", "Unknown"), o.get("name", "Unknown")))
                else:
                    print(f"Warning: Skipping non-dictionary element in 'objects' list: {o}")

            counts = Counter(names_mapped)

            objs_sorted = sorted(objs, key=lambda x: x.get("id", 0))

            type_counter = {}
            desc_list = []

            for o in objs_sorted:
                if isinstance(o, dict):
                    name_orig = o.get("name", "Unknown")
                    name = OBJ_NAME_MAP.get(name_orig, name_orig)
                    color_orig = o.get("color", "")
                    color = COLOR_MAP.get(color_orig, color_orig)
                    bbox = o.get("bbox", None)

                    pos_text = ""
                    size_text = ""

                    if bbox:
                        x_min, y_min, x_max, y_max = bbox
                        w = x_max - x_min
                        h = y_max - y_min
                        area_ratio = (w * h) / (img_w * img_h)

                        cx = (x_min + x_max) / 2
                        cy = (y_min + y_max) / 2
                        horiz = "trái" if cx < img_w/3 else "phải" if cx > 2*img_w/3 else "giữa"
                        vert = "trên" if cy < img_h/3 else "dưới" if cy > 2*img_h/3 else "giữa"
                        pos_text = f"ở {horiz}-{vert}"

                        if area_ratio > 0.2: size_text = "rất to"
                        elif area_ratio > 0.05: size_text = "to"
                        elif area_ratio > 0.01: size_text = "vừa"
                        else: size_text = "nhỏ"

                    idx = type_counter.get(name, 0) + 1
                    type_counter[name] = idx

                    order_text = number_to_vietnamese_order(idx)

                    desc = f"{name} {order_text} ({counts[name]})"
                    if color: desc += f" màu {color}"
                    if size_text: desc += f" ({size_text})"
                    if pos_text: desc += f" {pos_text}"
                    desc_list.append(desc)
                else:
                     print(f"Warning: Skipping non-dictionary element in 'objects_sorted' list: {o}")


            objects_text = ", ".join(desc_list)
    else:
        # Handle cases where 'objects' is neither a dict nor a list
        objects_text = ""
        print(f"Warning: 'objects' is not a dictionary or list: {objects_data}")


    combined = f"""
Tiêu đề: {metadata.get("title", "")}
Tiếng nói: {metadata.get("asr_text", "")}
Phân loại tiếng: {metadata.get("sound", "")}
Các vật thể: {objects_text}
""".strip()

    # print(combined) # Comment out print to avoid verbose output
    return combined
df["embedding_text"] = df.metadata.apply(build_text_for_embedding)

In [24]:
import pandas as pd

# Set display option to show full column width
pd.set_option('display.max_colwidth', None)

# Display the embedding_text column
display(df['embedding_text'])
df['embedding_text']

,embedding_text
0,"Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: \nPhân loại tiếng: Music:0.84;Choir:0.02;Singing:0.01\nCác vật thể: Đèn lồng: 2, Tòa nhà chọc trời: 5"
1,"Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: \nPhân loại tiếng: Music:0.84;Choir:0.02;Singing:0.01\nCác vật thể: Thuyền: 2, Clothing: 3, Người: 3, Tòa nhà chọc trời: 1"
2,"Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: chào mừng quý vị đến với chương trình sáu mươi giây của đài truyền hình thành phố hồ chí minh chương trình sét nay có những thông tin nổi bật sau đây\nPhân loại tiếng: Speech:1.00;Narration, monologue:0.01;Inside, small room:0.01\nCác vật thể: Clothing: 3, Người: 1, Đàn ông: 1, Phụ nữ: 1"
3,Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: \nPhân loại tiếng: \nCác vật thể: Pumpkin: 1
4,"Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: đường băng sông cửu long của tình trạng sục lúng gấp gần hai mươi lần so với nước biệt nhợt vận kệ ca tóc trái tim từ hà nội về vế ghép cho mệch nhân\nPhân loại tiếng: Speech:0.62;Music:0.46;Television:0.01\nCác vật thể: Ngôi nhà: 4, Cây: 6, Plant: 1"
...,...
302,"Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: vụ tai nạn xảy ra trên đoạn đường đông phương tiện giao thông dẫn đến tình trạng un tắc kéo dài lực lượng chức năng tiếp tục điều tra nguyên nhân và xử lý theo quy định\nPhân loại tiếng: Speech:0.87;Silence:0.01;Inside, small room:0.01\nCác vật thể: Van: 1, Xe tải: 1, Wheel: 1, Tire: 1"
303,"Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: vụ tai nạn xảy ra trên đoạn đường đông phương tiện giao thông dẫn đến tình trạng un tắc kéo dài lực lượng chức năng tiếp tục điều tra nguyên nhân và xử lý theo quy định\nPhân loại tiếng: Speech:0.87;Silence:0.01;Inside, small room:0.01\nCác vật thể: Người: 3, Land vehicle: 2, Clothing: 2, Đàn ông: 1"
304,"Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: vụ tai nạn xảy ra trên đoạn đường đông phương tiện giao thông dẫn đến tình trạng un tắc kéo dài lực lượng chức năng tiếp tục điều tra nguyên nhân và xử lý theo quy định\nPhân loại tiếng: Speech:0.87;Silence:0.01;Inside, small room:0.01\nCác vật thể: Người: 3, Clothing: 3, Human face: 1, Cậu bé: 1"
305,"Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: những thông tin vu rô xin được kết thúc chương trình sáu mươi giây sáng hôm nay của chúng tôi hẹn gặp lại quý vị trong chương trình sáu mươi giây chiều nay cà bây giờ xin kính chào tạm biệt\nPhân loại tiếng: Speech:1.00;Narration, monologue:0.01;Conversation:0.01\nCác vật thể: Clothing: 2, Đàn ông: 1, Người: 1"


,embedding_text
0,"Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: \nPhân loại tiếng: Music:0.84;Choir:0.02;Singing:0.01\nCác vật thể: Đèn lồng: 2, Tòa nhà chọc trời: 5"
1,"Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: \nPhân loại tiếng: Music:0.84;Choir:0.02;Singing:0.01\nCác vật thể: Thuyền: 2, Clothing: 3, Người: 3, Tòa nhà chọc trời: 1"
2,"Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: chào mừng quý vị đến với chương trình sáu mươi giây của đài truyền hình thành phố hồ chí minh chương trình sét nay có những thông tin nổi bật sau đây\nPhân loại tiếng: Speech:1.00;Narration, monologue:0.01;Inside, small room:0.01\nCác vật thể: Clothing: 3, Người: 1, Đàn ông: 1, Phụ nữ: 1"
3,Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: \nPhân loại tiếng: \nCác vật thể: Pumpkin: 1
4,"Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: đường băng sông cửu long của tình trạng sục lúng gấp gần hai mươi lần so với nước biệt nhợt vận kệ ca tóc trái tim từ hà nội về vế ghép cho mệch nhân\nPhân loại tiếng: Speech:0.62;Music:0.46;Television:0.01\nCác vật thể: Ngôi nhà: 4, Cây: 6, Plant: 1"
...,...
302,"Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: vụ tai nạn xảy ra trên đoạn đường đông phương tiện giao thông dẫn đến tình trạng un tắc kéo dài lực lượng chức năng tiếp tục điều tra nguyên nhân và xử lý theo quy định\nPhân loại tiếng: Speech:0.87;Silence:0.01;Inside, small room:0.01\nCác vật thể: Van: 1, Xe tải: 1, Wheel: 1, Tire: 1"
303,"Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: vụ tai nạn xảy ra trên đoạn đường đông phương tiện giao thông dẫn đến tình trạng un tắc kéo dài lực lượng chức năng tiếp tục điều tra nguyên nhân và xử lý theo quy định\nPhân loại tiếng: Speech:0.87;Silence:0.01;Inside, small room:0.01\nCác vật thể: Người: 3, Land vehicle: 2, Clothing: 2, Đàn ông: 1"
304,"Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: vụ tai nạn xảy ra trên đoạn đường đông phương tiện giao thông dẫn đến tình trạng un tắc kéo dài lực lượng chức năng tiếp tục điều tra nguyên nhân và xử lý theo quy định\nPhân loại tiếng: Speech:0.87;Silence:0.01;Inside, small room:0.01\nCác vật thể: Người: 3, Clothing: 3, Human face: 1, Cậu bé: 1"
305,"Tiêu đề: 60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024\nTiếng nói: những thông tin vu rô xin được kết thúc chương trình sáu mươi giây sáng hôm nay của chúng tôi hẹn gặp lại quý vị trong chương trình sáu mươi giây chiều nay cà bây giờ xin kính chào tạm biệt\nPhân loại tiếng: Speech:1.00;Narration, monologue:0.01;Conversation:0.01\nCác vật thể: Clothing: 2, Đàn ông: 1, Người: 1"


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Sample Text Data
corpus = df.embedding_text.tolist()

# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and Transform
vectorizer.fit_transform(corpus)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 24258 stored elements and shape (307, 1302)>

In [26]:
def get_sparse_embedding(text):
  # Transform Text into TF-IDF Sparse Vector
    tfidf_vector = vectorizer.transform([text])

    # Create Sparse Embedding for the New Text
    values = []
    dims = []
    for i, tfidf_value in enumerate(tfidf_vector.data):
        values.append(float(tfidf_value))
        dims.append(int(tfidf_vector.indices[i]))
    return {"values": values, "dimensions": dims}

### Create item

In [27]:
items = []
for i in range(len(df)):
    id = i
    sparse_embedding = get_sparse_embedding(df.embedding_text[i])
    items.append(
        {
            "id": id,
            "embedding": df['embedding'][i],
            "sparse_embedding": sparse_embedding,
            "embedding_metadata": {
                "asr_text": df.metadata[i].get('asr_text'),
                "sound": df.metadata[i].get('sound'),
                "objects": df.metadata[i].get('objects'),
                "image_url": df.metadata[i].get('image_url'),
                "timestamp": df.metadata[i].get('timestamp'),
                "frame_idx": df.metadata[i].get('frame_idx'),
                "title": df.metadata[i].get('title'),
            }
            })
items[:5]

[{'id': 0,
  'embedding': [-1.048385739326477,
   -1.274057269096374,
   -1.063393115997314,
   -0.129251584410667,
   2.955327987670898,
   -0.055299039930105,
   3.306627988815307,
   4.734584331512451,
   -3.748156547546386,
   2.174575567245483,
   -0.5926684737205501,
   -2.739755630493164,
   4.976145267486572,
   -0.754851758480072,
   -4.84751033782959,
   4.964186191558838,
   1.561791539192199,
   2.384532690048217,
   2.681716203689575,
   2.343520879745483,
   -2.596043348312378,
   0.9320942163467401,
   -3.100138425827026,
   -0.46165132522583,
   2.136367082595825,
   3.660256385803222,
   -3.038579702377319,
   -2.267643690109253,
   3.008506059646606,
   1.207713603973388,
   4.183225631713867,
   2.967486381530761,
   -3.712371587753296,
   -0.599265038967132,
   -1.601697087287902,
   -0.6329985857009881,
   1.607453465461731,
   -0.050787921994924004,
   0.48998603224754306,
   2.387729167938232,
   0.45560199022293,
   -2.703056097030639,
   -3.076615333557129,
   

### Upload file to GCS bucket

In [28]:
BUCKET_URI ="gs://test-video-retrieval/item"
# output as a JSONL file and save to the GCS bucket
with open("items.json", "w") as f:
    for item in items:
        f.write(f"{item}\n")
! gsutil cp items.json $BUCKET_URI


Copying file://items.json [Content-Type=application/json]...
/
Operation completed over 1 objects/6.8 MiB.                                      


## Build and Deploy a Vector Search Index

### Creat Index

In [29]:
# init the aiplatform package
from google.cloud import aiplatform

LOCATION = "asia-southeast1"
aiplatform.init(project="gen-lang-client-0974620078", location=LOCATION)

In [30]:
# create Index
INDEX_NAME = f"ai-challenge-text-{UID}"

my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=INDEX_NAME,    # index name
    dimensions=1024,
    contents_delta_uri='gs://test-video-retrieval/item',
    approximate_neighbors_count=20, # how many similar items we want to retrieve in typical cases
)

In [31]:
# Get exist Index
my_index = aiplatform.MatchingEngineIndex(index_name='8158086007028187136')

### Create Index Endpoint and deploy the Index

### Create endpoint

In [32]:
# create IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"ai-challenge-text-index-endpoint-{UID}", public_endpoint_enabled=True
)

### Deploy Index

In [33]:
DEPLOYED_INDEX_ID = f"ai_challenge_text_deployed_{UID}"
# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID)

resource name: projects/284454080854/locations/asia-southeast1/indexEndpoints/2339039464279506944

## Run a Query

In [40]:

from google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint import (
    HybridQuery,
)
# create HybridQuery
query_text = "có 1 người phụ nữ"
query_dense_emb = get_dense_embedding(query_text)
query_sparse_emb = get_sparse_embedding(query_text)
query = HybridQuery(
    dense_embedding=query_dense_emb,
    sparse_embedding_dimensions=query_sparse_emb["dimensions"],
    sparse_embedding_values=query_sparse_emb["values"],
    rrf_ranking_alpha=0.5,
)

In [41]:
# run a hybrid query
response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=[query],
    num_neighbors=10,
)

# print results
if response and response[0]:
    for idx, neighbor in enumerate(response[0]):
        # Access the title from the metadata dictionary using the neighbor.id as index
        title = df.loc[int(neighbor.id)]['metadata']['title']
        url = df.loc[int(neighbor.id)]['metadata']['image_url']
        dense_dist = neighbor.distance if neighbor.distance else 0.0
        sparse_dist = neighbor.sparse_distance if neighbor.sparse_distance else 0.0
        print(f"{title:<40}: dense_dist: {dense_dist:.3f}, sparse_dist: {sparse_dist:.3f}, image_url: {url}")
else:
    print("No neighbors found for the given query.")

60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024: dense_dist: 3316.538, sparse_dist: 0.000, image_url: https://storage.googleapis.com/test-video-retrieval/dataset/unzips/Keyframes_L21/keyframes/L21_V001/163.jpg
60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024: dense_dist: 0.000, sparse_dist: 0.493, image_url: https://storage.googleapis.com/test-video-retrieval/dataset/unzips/Keyframes_L21/keyframes/L21_V001/292.jpg
60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024: dense_dist: 0.000, sparse_dist: 0.397, image_url: https://storage.googleapis.com/test-video-retrieval/dataset/unzips/Keyframes_L21/keyframes/L21_V001/116.jpg
60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024: dense_dist: 3296.987, sparse_dist: 0.000, image_url: https://storage.googleapis.com/test-video-retrieval/dataset/unzips/Keyframes_L21/keyframes/L21_V001/287.jpg
60 Giây Sáng - Ngày 01082024 - HTV Tin Tức Mới Nhất 2024: dense_dist: 0.000, sparse_dist: 0.377, image_url: https://storage.google